# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [64]:
import numpy as np 
import pandas as pd
import math
from scipy.stats import percentileofscore as score
import requests
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [4]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [11]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
# data.status.code
data

{'companyName': 'Apple Inc',
 'marketcap': 2024960316561,
 'week52high': 151.6,
 'week52low': 55.09,
 'week52highSplitAdjustOnly': 146.97,
 'week52lowSplitAdjustOnly': 55.18,
 'week52change': 1.1541207093928931,
 'sharesOutstanding': 16947746188,
 'float': 0,
 'avg10Volume': 127442552,
 'avg30Volume': 133060720,
 'day200MovingAvg': 126.72,
 'day50MovingAvg': 127.84,
 'employees': 153397,
 'ttmEPS': 3.72,
 'ttmDividendRate': 0.8563663893537463,
 'dividendYield': 0.006890472592351202,
 'nextDividendDate': '0',
 'exDividendDate': '2021-02-01',
 'nextEarningsDate': '2021-01-17',
 'peRatio': 32.39460384079838,
 'beta': 1.17582631760541,
 'maxChangePercent': 45.13787585585228,
 'year5ChangePercent': 3.999033636170347,
 'year2ChangePercent': 1.5392332569046445,
 'year1ChangePercent': 1.147757337192201,
 'ytdChangePercent': -0.09684008727963955,
 'month6ChangePercent': 0.09799444503039517,
 'month3ChangePercent': -0.06569718607825108,
 'month1ChangePercent': -0.07763824033831748,
 'day30Change

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [12]:
data['year1ChangePercent']

1.147757337192201

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [15]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])


my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [29]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    # print(data.status_code) remove .json()
    
    # append dataframe to every ticker
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,127.70,0.890499,N/A
1,AAL,23.98,1.46716,N/A
2,AAP,191.68,1.56365,N/A
3,AAPL,126.72,1.14469,N/A
4,ABBV,110.30,0.595839,N/A
...,...,...,...,...
500,YUM,113.05,0.914847,N/A
501,ZBH,167.60,0.897279,N/A
502,ZBRA,465.05,1.62094,N/A
503,ZION,55.81,1.33873,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [34]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,467,VIAC,103.04,7.44907,N/A
1,179,FCX,35.12,5.59245,N/A
2,275,LB,60.58,5.44104,N/A
3,462,URI,313.87,3.59498,N/A
4,212,HAL,22.35,3.49862,N/A
5,305,MGM,39.25,3.45054,N/A
6,208,GPS,31.60,3.39957,N/A
7,175,FANG,77.18,3.24211,N/A
8,135,DISCA,76.15,3.2396,N/A
9,272,KSS,61.61,3.04816,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [38]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 10000000


10000000


In [43]:
# calculate position size
position_size = float(portfolio_size)/len(final_dataframe.index)

for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe

C:\Users\vyle4\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,467,VIAC,103.04,7.44907,1940
1,179,FCX,35.12,5.59245,5694
2,275,LB,60.58,5.44104,3301
3,462,URI,313.87,3.59498,637
4,212,HAL,22.35,3.49862,8948
5,305,MGM,39.25,3.45054,5095
6,208,GPS,31.60,3.39957,6329
7,175,FANG,77.18,3.24211,2591
8,135,DISCA,76.15,3.2396,2626
9,272,KSS,61.61,3.04816,3246


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [83]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
        ],
        index=hqm_columns),
        ignore_index=True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.68,N/A,0.891272,N/A,0.260805,N/A,0.0434599,N/A,-0.03397,N/A,N/A
1,AAL,24.28,N/A,1.44894,N/A,1.05186,N/A,0.573373,N/A,0.346925,N/A,N/A
2,AAP,189.38,N/A,1.51341,N/A,0.258242,N/A,0.162401,N/A,0.127687,N/A,N/A
3,AAPL,126.37,N/A,1.15199,N/A,0.0952447,N/A,-0.0652971,N/A,-0.0774173,N/A,N/A
4,ABBV,106.70,N/A,0.585171,N/A,0.195792,N/A,0.00891618,N/A,-0.0155228,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.24,N/A,0.916384,N/A,0.224114,N/A,-0.00146227,N/A,0.0278439,N/A,N/A
501,ZBH,162.26,N/A,0.911105,N/A,0.199975,N/A,0.0832363,N/A,0.0230033,N/A,N/A
502,ZBRA,472.96,N/A,1.62641,N/A,0.866477,N/A,0.227819,N/A,-0.0639621,N/A,N/A
503,ZION,56.60,N/A,1.3403,N/A,0.961903,N/A,0.349046,N/A,0.0930976,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [101]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0
            
for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe        

0        1
1     0.88
2     0.86
3     0.98
4     0.82
5      0.9
6     0.92
7     0.54
8     0.76
9     0.58
10    0.74
11    0.56
12    0.64
13    0.72
14     0.8
15    0.84
16    0.22
17     0.4
18    0.52
19    0.42
20    0.62
21    0.68
22    0.18
23    0.26
24    0.48
25     0.1
26     0.5
27    0.94
28    0.96
29    0.24
30     0.7
31    0.66
32    0.78
33     0.6
34    0.06
35     0.3
36    0.16
37    0.12
38    0.08
39    0.34
40    0.28
41    0.14
42    0.44
43    0.32
44     0.2
45    0.02
46    0.04
47    0.46
48    0.36
49    0.38
Name: One-Year Return Percentile, dtype: object
0     0.98
1        1
2     0.96
3     0.78
4     0.92
5      0.8
6     0.56
7     0.82
8      0.9
9     0.54
10     0.7
11    0.88
12    0.66
13    0.42
14    0.76
15    0.86
16    0.72
17    0.32
18    0.58
19    0.84
20     0.5
21    0.24
22    0.74
23     0.3
24    0.22
25    0.52
26     0.4
27    0.62
28    0.44
29    0.34
30     0.2
31    0.68
32    0.04
33     0.6
34     0.1
35    0.36
36    

C:\Users\vyle4\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,105.30,189,7.59083,1,2.47805,0.98,1.86042,1,0.571792,1,99.9505
1,DISCA,76.50,261,3.14319,0.88,2.55871,1,1.85872,0.98,0.534841,0.96,99.4554
2,DISCK,65.66,304,3.02196,0.86,2.25855,0.96,1.79925,0.96,0.548919,0.98,99.3069
3,LB,61.62,324,5.31651,0.98,1.08867,0.78,0.581989,0.86,0.209883,0.76,98.1188
4,KSS,60.21,332,2.95087,0.82,1.93501,0.92,0.62923,0.88,0.176028,0.7,97.9208
5,FANG,78.69,254,3.32843,0.9,1.46563,0.8,0.699036,0.92,0.165296,0.6,97.5743
6,GPS,30.40,657,3.45549,0.92,0.918175,0.56,0.536815,0.76,0.293974,0.9,97.3762
7,MRO,11.11,1800,2.12817,0.54,1.48903,0.82,0.709902,0.94,0.16638,0.62,96.4356
8,TPR,43.67,457,2.84376,0.76,1.76289,0.9,0.541335,0.78,0.138057,0.46,96.3861
9,NWSA,27.86,717,2.14962,0.58,0.909141,0.54,0.55478,0.8,0.222991,0.78,95.7921


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [102]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe                                 

C:\Users\vyle4\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,105.30,189,7.59083,1,2.47805,0.98,1.86042,1,0.571792,1,0.995
1,DISCA,76.50,261,3.14319,0.88,2.55871,1,1.85872,0.98,0.534841,0.96,0.955
2,DISCK,65.66,304,3.02196,0.86,2.25855,0.96,1.79925,0.96,0.548919,0.98,0.94
3,LB,61.62,324,5.31651,0.98,1.08867,0.78,0.581989,0.86,0.209883,0.76,0.845
4,KSS,60.21,332,2.95087,0.82,1.93501,0.92,0.62923,0.88,0.176028,0.7,0.83
5,FANG,78.69,254,3.32843,0.9,1.46563,0.8,0.699036,0.92,0.165296,0.6,0.805
6,GPS,30.40,657,3.45549,0.92,0.918175,0.56,0.536815,0.76,0.293974,0.9,0.785
7,MRO,11.11,1800,2.12817,0.54,1.48903,0.82,0.709902,0.94,0.16638,0.62,0.73
8,TPR,43.67,457,2.84376,0.76,1.76289,0.9,0.541335,0.78,0.138057,0.46,0.725
9,NWSA,27.86,717,2.14962,0.58,0.909141,0.54,0.55478,0.8,0.222991,0.78,0.675


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [103]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

<ipython-input-103-0b4c9a2eb497>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,105.30,189,7.59083,1,2.47805,0.98,1.86042,1,0.571792,1,0.995
1,DISCA,76.50,261,3.14319,0.88,2.55871,1,1.85872,0.98,0.534841,0.96,0.955
2,DISCK,65.66,304,3.02196,0.86,2.25855,0.96,1.79925,0.96,0.548919,0.98,0.94
3,LB,61.62,324,5.31651,0.98,1.08867,0.78,0.581989,0.86,0.209883,0.76,0.845
4,KSS,60.21,332,2.95087,0.82,1.93501,0.92,0.62923,0.88,0.176028,0.7,0.83
5,FANG,78.69,254,3.32843,0.9,1.46563,0.8,0.699036,0.92,0.165296,0.6,0.805
6,GPS,30.40,657,3.45549,0.92,0.918175,0.56,0.536815,0.76,0.293974,0.9,0.785
7,MRO,11.11,1800,2.12817,0.54,1.48903,0.82,0.709902,0.94,0.16638,0.62,0.73
8,TPR,43.67,457,2.84376,0.76,1.76289,0.9,0.541335,0.78,0.138057,0.46,0.725
9,AAL,24.28,823,1.44894,0.22,1.05186,0.72,0.573373,0.84,0.346925,0.92,0.675


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [104]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [105]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,105.30,189,7.59083,1,2.47805,0.98,1.86042,1,0.571792,1,0.995
1,DISCA,76.50,261,3.14319,0.88,2.55871,1,1.85872,0.98,0.534841,0.96,0.955
2,DISCK,65.66,304,3.02196,0.86,2.25855,0.96,1.79925,0.96,0.548919,0.98,0.94
3,LB,61.62,324,5.31651,0.98,1.08867,0.78,0.581989,0.86,0.209883,0.76,0.845
4,KSS,60.21,332,2.95087,0.82,1.93501,0.92,0.62923,0.88,0.176028,0.7,0.83
5,FANG,78.69,254,3.32843,0.9,1.46563,0.8,0.699036,0.92,0.165296,0.6,0.805
6,GPS,30.40,657,3.45549,0.92,0.918175,0.56,0.536815,0.76,0.293974,0.9,0.785
7,MRO,11.11,1800,2.12817,0.54,1.48903,0.82,0.709902,0.94,0.16638,0.62,0.73
8,TPR,43.67,457,2.84376,0.76,1.76289,0.9,0.541335,0.78,0.138057,0.46,0.725
9,AAL,24.28,823,1.44894,0.22,1.05186,0.72,0.573373,0.84,0.346925,0.92,0.675


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [106]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [107]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [108]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [109]:
writer.save()